#### 데이터 분석

- 문제 정의 = 강의 컨텐츠가 개편되어 8월에 적용되었다. 강의 컨텐츠 개선이 수강생들에게 긍정적인 영향을 미쳤는가?
- 가설 설정 = 강의 컨텐츠 개편이 부정적이라면 개편된 강의를 수강한 수강생들의 완주율이 떨어졌을 것이다.
- 데이터 분석 기본 세팅
- 데이터 분석 = 
- 분석 결과 시각화
- 최종 결론
    - 강의 컨텐츠 개편이 부정적 영향이 아닌 8월 전체 완주율이 떨어 진것으로 확인되었습니다.
    - 다른 요인 때문에 완주율이 떨어졌다고 보는 것이 맞을 것 같습니다
    
#### 좋은 가설이란?

- 풀고자 하는 문제의 방향성과 일치하는 가설 → 무슨 문제를 풀고자 하는지 명확히 알 것
- 테스트 가능한 가설 → 데이터로 확인이 가능한 가설을 세울 것
- 액션으로 이어질 수 있는 가설 → 가설 검증 후 결과에 따라 특정 액션으로 유도될 수 있을 것

#### 코호트 차트란?

- 소비자를 시간의 흐름과 사용하는 서비스의 단계 등으로 나누어 같은 성질의 집단으로 구분한 것
- 소비자의 전환율을 시간의 흐름과 서비스의 단계나 재사용 등 두 가지 요인을 복합적으로 보여줄 수 있는 것

#### 코호트 분석 방법

- 각 고객별 최초 수강 주(week) 구하기
- 진도율을 강의 주차로 변경 하기
- 데이터 분석하기
    - 수강 시작 주와, 수강 주차에 각 해당하는 수강생 수를 내용으로 테이블 만들기
    - 수강 주차 별, 수강한 총 인원 구하기
- 데이터 시각화하기
    - 고객 수강 유지율을 알수 있는 피벗 테이블 구하기
        - 아래 세 가지를 한 눈에 볼 수 있는 피벗테이블 구하기 
            - 행 →  수강 시작 주차 
            - 열 →  수강 주차
            - 행과 열사이의 정보 → 수업을 들은 수강생 수
    - 각 수강 시작 주(week)당 초기 고객 수 구하기
    - 피벗테이블의 각 데이터를 초기 고객수로 나눠서, 퍼센테이지로 나타내기
    
- 히트맵으로 특정 기간동안 고객의 수강율 나타내기
- 결론 도출하기

In [ ]:
# Pandas 라이브러리 불러오기

import pandas as pd

# 수강생 수강 파일 불러오기

sparta_data = pd.read_table('../Data/cohort_data.csv',sep=',')
sparta_data.head()

In [ ]:
# 시각화 라이브러리 불러오기

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# 글씨 깨짐방지

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# 폰트 설정

plt.rc('font', family='Malgun Gothic')

In [ ]:
# 파이썬의 type()함수를 쓰면, 데이터의 종류를 확인 할수 있어요 :)
# sparta_date 정보에서 접속 날짜 열에서 데이터 첫번째 부분만 확인 하면 되겠죠?

print(type(sparta_data['created_at'][1]))

In [ ]:
# 문자열 데이터를 시계열 데이터로 변경 후 파생변수 생성
# format='%Y-%m-%dT%H:%M:%S.%f'
# sparta_data['start_time'] = pd.to_datetime(sparta_data['created_at'], format=format,infer_datetime_format=True)
# sparta_data.tail()

sparta_data['start_time'] = pd.to_datetime(sparta_data['created_at'], format='mixed')
sparta_data.tail()

In [ ]:
# 수강 시작 주(week) 열 추가하기
# 수강 시작 주 구하기(1년 중 몃주차 인지)

sparta_data['start_week']= sparta_data['start_time'].dt.isocalendar().week # day
sparta_data.tail()

In [ ]:
# 처음 수강 시작한 주 범위 확인 하기
# 이전에 배웠듯이 set()은 set안의 데이터는 순서가 정해져있지 않고, 중복되지 않는 고유한 요소를 가져옵니다!
# 몃개의 개강반이 있는지 확인

category_range = set(sparta_data['start_week'])
category_range

In [ ]:
# 범주화 할 데이터 리스트로 만들기

progress_rate = list(sparta_data['progress_rate'])
progress_rate

In [ ]:
# 수강주차 레이블 만들기
# 0주차 : 0 ~4 .11%
# 1주차 : 4.12% ~ 26.03%
# 2주차 : 26.04% ~ 41.10%
# 3주차 : 41.11% ~ 61.64%
# 4주차 : 61.65% ~ 80.82%
# 5주차 : 80.83% ~ 100%
# 범주를 구분하는 기준 bins 처음(0)과 끝(100) 잊지 말고 기입 해주세요!

bins = [0,4.11,26.03,41.10,61.64,80.82,100]

# 구분한 범주의 라벨 labels

labes=[0,1,2,3,4,5]

# 범주화 하기
# 범주화에 사용하는 함수 pd.cut

cuts = pd.cut(progress_rate,bins, right=True,include_lowest=True, labels=labes)
cuts

In [ ]:
# 범주화 결과물을 테이블로 변경하기

cuts = pd.DataFrame(cuts)
cuts.head()

In [ ]:
# 기존 테이블에 현재 수강 주차 테이블 합치기
# concat() 함수를 이용하여, sparta_data 테이블과, cuts 테이블 병합 할수 있습니다 :)
# index를 기준으로 합친다

sparta_data = pd.concat([sparta_data,cuts],axis=1, join='inner')
sparta_data.head()

In [ ]:
# 컬럼이름 변경 0 -> week
# 그래서, 귀찮더라도, 우리가 원하는 컬럼의 이름을 다 작성해 줍시다!

sparta_data.columns=['created_at','user_id','name','progress_rate','start_time','start_week','week']
sparta_data.head()

In [ ]:
# 수강 시작 주와, 수강 주차를 기준으로 테이블 만들기
# 그룹화
# 기존의 테이블을, start_week와, week로 묶어줍니다!

grouping = sparta_data.groupby(['start_week','week'])
grouping.head()

In [ ]:
# 수강 시작 주와, 수강 주차에 각 해당하는 수강생 수 구하고, 테이블로 변경하기 

cohort_data = grouping['user_id'].apply(pd.Series.nunique)
cohort_data = pd.DataFrame(cohort_data)
cohort_data.head(20)

In [ ]:
# 수강 주차 별, 수강한 총 인원 구하기
# 각 주차별에 머물러 있는 수강생 수는 구했지만, 
# 우리에게 필요한 것은 수강 주차별 수강을 완료한 총 인원 입니다!
# 수강 주차 별 수강한 총 인원 구하기
# 첫 주가 31주니 변수를 하나 만들어 줍니다!

f=31

# 처음 수강 시작한 주의 범위가 {31,32,33,34,35,36} 이니, range(6)으로 합시다!

for i in range(6):
	# 5주차의 강의가 마지막이고, 0주차까지 이니, 시작은 5에서 시작해 1씩 0까지 감소 시킬수 있어요!
  for j in range(5, 0, -1):
    cohort_data.at[(f,j-1), 'user_id'] = int(cohort_data.at[(f,j),'user_id']) +  int(cohort_data.at[(f,j-1),'user_id'])
	# 주차는(31부터 32 33..) 1씩 늘어나죠?
  f=f+1

cohort_data.head(20)

In [ ]:
# cohort_data에 인덱스 설정하기

cohort_data = cohort_data.reset_index()
cohort_data.head(20)

In [ ]:
# 피벗테이블 만들기

cohort_counts = cohort_data.pivot(index="start_week",
                                  columns="week",
                                  values="user_id")
cohort_counts

In [ ]:
# 좀 더 가시화 시키기! 표의 데이터를 퍼센테이지로 나타내 봅시다!
# 🤔 에?? 또.. 테이블을 바꾼다고요? 왜요..?
# 수강 총 인원을 퍼센테이지로 변경한 테이블이 필요한 이유
# 우리가 고객의 “수”로 고객 수강 이탈을 판단하는건 쉽지 않아요!
# 개강일별로 전체 인원이 모두 다르기 때문입니다!
# 그래서, 고객의 행동이 얼만큼 변화가 있는지에 대한 부분을 비율로 나타내면 
# 좀 더 한 눈에 알아 볼수 있는 분석 결과가 됩니다!
# 이것을 리텐션 테이블이라고 합니다
# 리텐션 테이블에 대해서 알아볼까요?
# 고객이 우리 제품이나 서비스를 지속적으로 소비하는 것을 의미 합니다! 
# 그러니 이것을 수치화해 기간을 나눠 테이블로 나타내면 지속적으로 고객이 우리의 서비스를 사용하는지 여부를 쉽게 알 수 있겠죠?😀

# 리텐션 테이블 생성 및 각 데이터에 나눠 줄 수강 시작 주 총 인원 구하기
# 앞서 만든 피벗 테이블을 retention 변수에 저장하기

retention = cohort_counts

# 각 주(week) 별 최초 수강생 수만 가져오기 (나눠줄때, 분모가 되는 부분!)
# 0주차 수강생 가져오기

cohort_sizes = cohort_counts.iloc[:,0]
cohort_sizes.head()

In [ ]:
# 각 데이터에 수강 시작 주의 총 인원 나눠주기
# 표의 단일 데이터에 최초 수강생의 수를 나누어, 각 주당 수강생 수강율 나타내기! 

retention = cohort_counts.divide(cohort_sizes, axis=0)
retention.head()

In [ ]:
# 각 수치 퍼센트로 변경하기
# round 함수로 3자리 수에서 반올림 한 후, 100을 곱해 줍니다!

retention.round(3)*100

In [ ]:
# 코호트 분석 히트맵으로 시각화 하기
# 히트맵 찍어보기

sns.heatmap(data=retention,
           annot=True, # 각 cell의 데이터 표기 유무를 나타냅니다!
           fmt='.0%', # values(데이터의 값) 값의 소수점 표기
           vmin=0, # 최소값 설정
           vmax=1, # 최댓값 설정
           cmap="BuGn" # 히트맵의 색을 설정합니다 
						)

In [ ]:
# 완성 코드 및 완성된 그래프
# 테이블 크기 설정 하기

plt.figure(figsize=(10,8))

sns.heatmap(data=retention,
           annot=True, # 각 cell의 데이터 표기 유무를 나타냅니다!
           fmt='.2%', # values(데이터의 값) 값의 소수점 표기
           vmin=0, # 최소값 설정
           vmax=1, # 최댓값 설정
           cmap="BuGn" # 히트맵의 색을 설정합니다
						)

plt.xlabel('주차', fontsize=14,labelpad=30)
plt.ylabel('개강일', fontsize=14,rotation=360,labelpad=30)
# y축 레이블 축 변경 하기
plt.yticks(rotation=360)
# 테이블 보여주기
plt.show()

In [ ]:
# 7강에서는 구하고 싶은 커리큘럼별 인원을 처음인원으로 나누었다면
# 이번에는 구하고 싶은 커리큘럼별 인원을 직전주차 인원로 나누어 개강일별 주차 간 전환율을 구할 수 있어요!

# 라이브러리 불러오기

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font',family='Malgun Gothic')

# 한글깨짐 방지

plt.rc('font',family='Malgun Gothic')
sparta_data = pd.read_table('./Data/cohort_data.csv',sep=',')
sparta_data.tail()

# 날짜 데이터 타입 변경

format='%Y-%m-%dT%H:%M:%S.%f'
sparta_data['start_time'] = pd.to_datetime(sparta_data['created_at'], format='mixed',infer_datetime_format=True)
sparta_data.tail()

# 시작 week 구하기

sparta_data['start_week']= sparta_data['start_time'].dt.isocalendar().week
sparta_data.tail()

# 시작 주 범위 알기

category_range = set(sparta_data['start_week'])
category_range

# 범주화 하기
# 범주화할 데이터 

progress_rate = list(sparta_data['progress_rate'])
progress_rate

# 범주를 구분하는 기준 bins

bins = [0,4.11,26.03,41.10,61.64,80.82,100]

# 구분한 범주의 라벨 labels

labes=[0,1,2,3,4,5]

# 범주화에 사용하는 함수 pd.cut

cuts = pd.cut(progress_rate,bins, right=True,include_lowest=True, labels=labes)
cuts
cuts = pd.DataFrame(cuts)
cuts.tail()

# 표 합치기
sparta_data = pd.concat([sparta_data,cuts],axis=1, join='inner')
sparta_data.head()

# 표 인덱스 변경하기

sparta_data.columns=['created_at','user_id','name','progress_rate','start_time','start_week',"week"]
sparta_data.head()

# 시작주와, 수강 주차별 기준으로 표 grouping 하기

grouping = sparta_data.groupby(['start_week','week'])
grouping.head()

# 시작주와, 수강 주차별에 해당하는 수강생 수 구하기

cohort_data = grouping['user_id'].apply(pd.Series.nunique)
cohort_data = pd.DataFrame(cohort_data)
cohort_data.head(10)

# 각 주차별 수강한 수강생 총 합 구하기

k=31
for i in range(6):
  for j in range(5, 0, -1):
    cohort_data.at[(k,j-1), 'user_id'] = int(cohort_data.at[(k,j),'user_id']) +  int(cohort_data.at[(k,j-1),'user_id'])
  k=k+1
cohort_data = cohort_data.reset_index()
cohort_data.head()

cohort_counts = cohort_data.pivot(index="start_week",
                                  columns="week",
                                  values="user_id")
cohort_counts

# 앞서 만든 피벗 테이블을 retention 변수에 저장하기

retention = cohort_counts

# 각 주(week) 별 최초 수강생 수만 가져오기

cohort_sizes = cohort_counts.iloc[:,0]
cohort_sizes.head()

# 최초 수강생 수를 각 데이터에 나눠주기

retention = cohort_counts.divide(cohort_sizes, axis=0)
retention

# 각 수치 퍼센트로 변경하기

retention.round(3)*100

# 주차별 수강 전환율 구하기

w=31
for i in range(6):
  for j in range(5, 0, -1):
    retention.at[(w,j)] = retention.at[(w,j)]/retention.at[(w,j-1)]
  w=w+1

retention

# 주차별 수강 전환율 히트맵

plt.figure(figsize=(10,8))

sns.heatmap(data=retention,
           annot=True,
           fmt='.0%',
           vmin=0,
           vmax=1,
           cmap="BuGn")



plt.title('개강일별 주차 간 전환율', fontsize=20)
plt.xlabel('주차', fontsize=14,labelpad=30)
plt.ylabel('개강일', fontsize=14,rotation=360,labelpad=30)
plt.yticks(rotation=360)

plt.show()